# SVM
Support vector machine（支持向量机）,是一种分类算法，它的特点在于找出的分类面是最优的。对于二维数据来说，它可以找到一条直线，这是无数条可以分类的直线当中最完美的，因为它恰好在两个类的中间，距离两个类的点都一样远（？存疑）。如果是高维的点，SVM的分界线就是平面或者超平面（hyperplane）。其算法效果可由下图所示：

![svm1](./resources/svm1.jpg)

上图中，Boundary代表决策面，Margin代表分类间隔，Support Vector代表离决策面最近的那些数据点（虚线穿过的点）。

需要注意的是：如果一个类别的数据点跑到了另一类别中，那么它就是该类别的离散值。SVM 的一个特征就是会忽略离散值并找到具有最大边距的超平面。因此，我们可以说，SVM 对于离散值具有鲁棒性。

SVM有核函数，该函数具有将低维数据转化成高维数据的作用，它可将不可分离的问题转换成可分离的问题。

**实际上最优决策面的方向和位置完全取决于选择哪些样本点作为支持向量**。

## 求解最优决策面

一个最优化问题通常有两个最基本的因素：
- 目标函数，也就是你希望什么东西的什么指标达到最好;
- 优化对象，你期望通过改变哪些因素来使你的目标函数达到最优;

在线性SVM算法中，目标函数是**分类间隔**，而优化对象则是**决策面**。

### Direct Representation

求解最优决策面最直接的表达形式如下所示，其中$margin$函数可以输入一个$boundary$，计算正确分类的所有苹果与香蕉到$boundary$的最小距离。

> $argmax_{boundary} \ margin(boundary)$  
> $s.t.$ 所有正确归类的苹果与香蕉到$boundary$的距离都$>=margin$

考虑使用数学表达来描述：

1.定义直线$x_2=ax_1+b$为决策面，经过变换可以得到：$\boldsymbol{\omega}^T\boldsymbol{x}+\gamma=0$，需要注意$\omega$控制了直线的方向。

2.平面上任意点$x_0$到直线的距离为：$\dfrac{1}{||\boldsymbol{\omega}||}(\boldsymbol{\omega}^Tx_0+\gamma)$；假设支持向量到决策面的距离为$d$，则所有样本点需要满足：$\left\{\begin{array}{ll} (\boldsymbol{\omega}^T\boldsymbol{x}_i+\gamma)/||\boldsymbol{\omega}||\geq d & \forall~ y_i=1\\(\boldsymbol{\omega}^T\boldsymbol{x}_i+\gamma)/||\boldsymbol{\omega}||\leq -d & \forall~y_i=-1\end{array}\right. $
，两边同除以$d$不影响原决策面的方向与截距，则SVM优化的约束条件变为：$\left\{\begin{array}{ll} \boldsymbol{\omega}^T\boldsymbol{x}_i+\gamma\geq 1 & \textrm{for} \  y_i=1\\\boldsymbol{\omega}^T\boldsymbol{x}_i+\gamma\leq -1 & \textrm{for} \ y_i=-1\end{array}\right. $。
需要注意的是：支持向量样本点到决策面方程的距离就是$1/||\boldsymbol{\omega}||$（等于1或者-1只发生在样本点为支持向量的时候），则问题转化为求$||\boldsymbol{\omega}||$最小值的问题。综上，SVM最优化问题的数学描述为：

> $\begin{array}{l} \min_{\boldsymbol{\omega},\gamma}\frac{1}{2}||\boldsymbol{\omega}||^2\\ \textrm{s. t.}~ ~y_i(\boldsymbol{\omega}^T\boldsymbol{x}_i+\gamma)\geq 1,~~i = 1,2,...,m \end{array}$

缩写s.t.表示“Subject to”，是“服从某某条件”的意思，之所以要在$||\boldsymbol{\omega}||$上加上平方和1/2的系数，是为了以后进行最优化的过程中对目标函数求导时比较方便,但这绝不影响最优化问题最后的解。于是，求解SVM最优决策面的问题转化成了一个**不等式约束条件下的优化问题**。

### 最优问题约束条件的几何意向
约束条件一般分为等式约束和不等式约束两种，前者表示为$g(\boldsymbol{x})=0$；后者表示为$g(\boldsymbol{x})\leq0$。

假设$\boldsymbol{x}\in\mathbb{R}^d$（就是这个向量一共有d个标量组成），则$g(\boldsymbol{x})=0$的几何意象就是d维空间中的d-1维曲面，如果函数$g(\boldsymbol{x})$是线性的，$g(\boldsymbol{x})=0$则是个d-1维的超平面。那么有约束优化问题就要求在这个d-1维的曲面或者超平面上找到能使得目标函数最小的点，这个d-1维的曲面就是“可行解区域”。

对于不等式约束条件，$g(\boldsymbol{x})\leq0$，则可行解区域从d-1维曲面扩展成为d维空间的一个子集。我们可以从d=2的二维空间进行对比理解。等式约束对应的可行解空间就是一条线；不等式约束对应的则是这条线以及线的某一侧对应的区域，就像下面这幅图的样子（图中的目标函数等高线其实就是等值线，在同一条等值线上的点对应的目标函数值相同）：

![svm2](./resources/svm2.jpg)

### 求解等式约束条件下最优解：拉格朗日乘子法
假设目标函数为$f(\boldsymbol{x})$，约束条件为$g(\boldsymbol{x})$。拉格朗日乘子法的基本思想是把约束条件转化为新的目标函数$L(\boldsymbol{x},\lambda)$的一部分，从而使有约束优化问题变成我们习惯的无约束优化问题。

对于等式约束条件下情况（上图左），我们观察一下图中的红色虚线（可行解空间）和蓝色虚线（目标函数的等值线），发现这个被约束的最优解恰好在二者相切的位置（证明从略）。则可以得到：$\nabla f(\boldsymbol{x}^*)+\lambda \nabla g(\boldsymbol{x}^*)=0$（因为两者的梯度在一条直线上），且$g(\boldsymbol{x}^*)=0$。

为了在数学上更加便捷和优雅一点，我们构造一个拉格朗日函数，将有约束优化问题转为无约束优化问题。拉格朗日函数具体形式如下：$L(\boldsymbol{x},\lambda)=f(\boldsymbol{x})+\lambda g(\boldsymbol{x})$,将公式分别对$\boldsymbol{x}$,$\lambda$求导，令结果等于零，就可以建立两个方程（就是前面的两个）。

### 获得不等式最优解约束条件：KKT条件
对于不等式约束条件$g(\boldsymbol{x})\leq0$的情况，如下图所示，最优解所在的位置$\boldsymbol{x}^*$有两种可能，或者在边界曲线$g(\boldsymbol{x})=0$上或者在可行解区域内部满足不等式$g(\boldsymbol{x})<0$的地方。

![svm3](./resources/svm3.jpg)

对于第一种情况则：$\lambda>0, \ g(x)=0$（$f(x)$梯度与$g(x)$梯度相反）；对于第二种情况：$\lambda=0, \ g(x)<0$（相当于约束条件没有起作用）。无论哪一种情况都有$\lambda g(\boldsymbol{x})=0$。所以可以得到KKT条件如下：

> $\begin{array}{lr} g(\boldsymbol{x})\leq0~ & (1)\\ \lambda\geq0~ & (2)\\ \lambda g(\boldsymbol{x})=0 & (3) \end{array}$

注意：**KKT条件是对最优解的约束，而原始问题中的约束条件是对可行解的约束**。

### 拉格朗日对偶

## 参考
1. https://www.zhihu.com/question/21094489/answer/86273196
2. [零基础学SVM—Support Vector Machine(一)](https://zhuanlan.zhihu.com/p/24638007)